In [94]:
import json
import pandas as pd
import os
import re
from tqdm.notebook import tqdm

In [95]:
def get_filepaths(rootdir='.', filetypes=['txt']):
    
    """
    Returns the path-string of all the json files in the given root dirctory.

    Usage:
        from utils import get_filepaths

        get_filepaths(rootdir='~/path/to/dir', filetypes=['txt'])

    Parameters:
        rootdir:
              The directory where the files are read from.
              The default is '.'

        filetypes:
              A list containing the string form of the file types to read.
              Default: ['txt']  reads all text files
              pass an empty list [] for reading all file types

    This function raises an exception if the given path does not exist in the local system.
    """
    # If found, remove the '/' at the end of rootdir 
    if rootdir[-1] == os.sep:
        rootdir = rootdir[:-1]

    # If the directory does not exist on the user system, then raise an exception error
    if os.path.exists(rootdir) is False:
        raise Exception(f"Directory `{rootdir}` does not exist.")

    # Go through the folder structure and add to filepaths list
    filepaths = []
    # Convert filetypes to lower case
    filetypes = [ftype.lower() for ftype in filetypes]

    for (dirpath, dirnames, filenames) in os.walk(rootdir):
        for filename in filenames:
            # if filename is in given filetypes
            if filetypes == []:
                filepaths.append(os.path.join(dirpath, filename))
            else:
                # Split the filename with . and check if it is the desired extension
                if filename.split('.')[-1].lower() in filetypes:
                    filepaths.append(os.path.join(dirpath, filename))

    # return the filepaths list
    return filepaths

In [96]:
filepaths = get_filepaths('./dataset1')
# filepaths.extend(get_filepaths('./dataset2'))

In [97]:
all_data = []

In [98]:
for fp in tqdm(filepaths):
    data = json.load(open(fp, 'r'), encoding = 'UTF-8')
    all_data.extend(data)    

In [99]:
JSON_df = pd.DataFrame(all_data)
JSON_df.head()

,ArrivalStationCode,CurrentDate,Date,DepartureStationCode,Flights,HasSelection,InMonth,MinimumPrice
0,BUD,26/10/2015,20151026,BCN,[],True,False,None
1,BUD,27/10/2015,20151027,BCN,[],True,False,None
2,BUD,28/10/2015,20151028,BCN,[],True,False,None
3,BUD,29/10/2015,20151029,BCN,[],True,False,None
4,BUD,30/10/2015,20151030,BCN,[],True,False,None


In [100]:
# Expand nested data from Flights column and drop rows with NA values
df = JSON_df.explode(column='Flights').dropna()
df.reset_index(inplace=True, drop=True)

In [101]:
# Delete Flights column and concatenate the newly generated columns to the main dataframe
df = pd.concat([df.drop(['Flights'], axis=1), df['Flights'].apply(pd.Series)], axis=1)
df.head()

,ArrivalStationCode,CurrentDate,Date,DepartureStationCode,HasSelection,InMonth,MinimumPrice,CarrierCode,FlightNumber,STD,STA,ArrivalStationName,DepartureStationName,IsMACStation,IsAirportChange
0,BUD,11/11/2015,20151111,BCN,True,True,â‚¬109.99,W6,2376,09:15,11:50,Budapest,Barcelona El Prat,True,False
1,BUD,13/11/2015,20151113,BCN,True,True,â‚¬89.99,W6,2376,09:15,11:50,Budapest,Barcelona El Prat,True,False
2,BUD,15/11/2015,20151115,BCN,True,True,â‚¬89.99,W6,2376,09:15,11:50,Budapest,Barcelona El Prat,True,False
3,BUD,16/11/2015,20151116,BCN,True,True,â‚¬49.99,W6,2376,09:15,11:50,Budapest,Barcelona El Prat,True,False
4,BUD,18/11/2015,20151118,BCN,True,True,â‚¬69.99,W6,2376,09:15,11:50,Budapest,Barcelona El Prat,True,False


In [102]:
# df.to_csv('airtickets.csv')

In [103]:
# df.info()

In [104]:
df['CurrentDate'] = pd.to_datetime(df['CurrentDate'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['STD'] = pd.to_datetime(df['STD'], errors='coerce')
df['STA'] = pd.to_datetime(df['STA'], errors='coerce')

In [105]:
df['HasSelection'] = df['HasSelection'].astype('bool')
df['InMonth'] = df['InMonth'].astype('bool')
df['IsMACstation'] = df['IsMACStation'].astype('bool')
df['IsAirportChange'] = df['IsAirportChange'].astype('bool')

In [106]:
df['FlightNumber'] = df['FlightNumber'].astype(str)
df['ArrivalStationCode'] = df['ArrivalStationCode'].astype(str)
df['DepartureStationCode'] = df['DepartureStationCode'].astype(str)
df['ArrivalStationName'] = df['ArrivalStationName'].astype(str)
df['DepartureStationName'] = df['DepartureStationName'].astype(str)

In [107]:
df['MinimumPrice'] = df['MinimumPrice'].astype('str')

In [108]:
# df.info()

In [109]:
# df['MinimumPrice'].value_counts()

In [110]:
def clean_currency(x):
    """ Remove comma in currency numeric
        Add space between currency symbol and value
    """
    if isinstance(x, int) or isinstance(x, float):
        return ('', x)
    if isinstance(x, str):
        numeric_start_at = 0
        for i in range(len(x)):
            if x[i].isnumeric():
                numeric_start_at = i
                break
        return (x[0:numeric_start_at], x[numeric_start_at:].replace(',', ''))
    return('', x)

In [111]:
df['MinimumPrice'] = df['MinimumPrice'].apply(clean_currency).astype('str')

In [112]:
df1 = df.head(1000).apply(lambda x: pd.Series(clean_currency(x['MinimumPrice']),
                                              index=['symbol', 'value']), axis=1, result_type='expand')

In [113]:
currency_correction_map = {
    "â‚¬": "EUR",
    "lei": "LEI",
    "Ft" : "FT",
    "Kr" : "KR",
    "MKD" : "MKD"
}

def currency_correction(x):
    if x in currency_correction_map:
        return currency_correction_map[x]
    else:
        return x

In [114]:
df1['symbol'] = df1['symbol'].apply(currency_correction)
df1

,symbol,value
0,"('â‚¬', '",109.99')
1,"('â‚¬', '",89.99')
2,"('â‚¬', '",89.99')
3,"('â‚¬', '",49.99')
4,"('â‚¬', '",69.99')
...,...,...
995,"('â‚¬', '",69.99')
996,"('â‚¬', '",69.99')
997,"('â‚¬', '",89.99')
998,"('â‚¬', '",89.99')


In [134]:
currency_multiplier_EUR = {
    "EUR": 1,
    "LEI": 0.21,
    "FT" : 0.0027,
    "KR" : 0.099,
    "MKD" : 0.016       
}

def currency_converter(symbol, value):
    value = value.strip("'")[0]
    value = float(value)
    if symbol in currency_multiplier_EUR:
        return currency_multiplier_EUR[symbol] * value
    else:
        return None

In [136]:
df1['Price_in_EUR'] = df1.apply(lambda x: currency_converter(x['symbol'], x['value']), axis=1)
df1.head()

,symbol,value,Price_in_EUR
0,"('â‚¬', '",109.99'),None
1,"('â‚¬', '",89.99'),None
2,"('â‚¬', '",89.99'),None
3,"('â‚¬', '",49.99'),None
4,"('â‚¬', '",69.99'),None


In [121]:
df2 = df.head(1000)
final_df = pd.concat([df2.drop(['MinimumPrice'], axis=1), df1], axis=1)

In [123]:
final_df.head()

,ArrivalStationCode,CurrentDate,Date,DepartureStationCode,HasSelection,InMonth,CarrierCode,FlightNumber,STD,STA,ArrivalStationName,DepartureStationName,IsMACStation,IsAirportChange,IsMACstation,symbol,value,Price_in_EUR
0,BUD,2015-11-11,2015-11-11,BCN,True,True,W6,2376,2021-01-02 09:15:00,2021-01-02 11:50:00,Budapest,Barcelona El Prat,True,True,True,"('â‚¬', '",109.99'),None
1,BUD,2015-11-13,2015-11-13,BCN,True,True,W6,2376,2021-01-02 09:15:00,2021-01-02 11:50:00,Budapest,Barcelona El Prat,True,True,True,"('â‚¬', '",89.99'),None
2,BUD,2015-11-15,2015-11-15,BCN,True,True,W6,2376,2021-01-02 09:15:00,2021-01-02 11:50:00,Budapest,Barcelona El Prat,True,True,True,"('â‚¬', '",89.99'),None
3,BUD,2015-11-16,2015-11-16,BCN,True,True,W6,2376,2021-01-02 09:15:00,2021-01-02 11:50:00,Budapest,Barcelona El Prat,True,True,True,"('â‚¬', '",49.99'),None
4,BUD,2015-11-18,2015-11-18,BCN,True,True,W6,2376,2021-01-02 09:15:00,2021-01-02 11:50:00,Budapest,Barcelona El Prat,True,True,True,"('â‚¬', '",69.99'),None
